Соединение и аутентификация

При удачном подключении к почте увидим сообщение [b'Authentication successful']

In [36]:
import imaplib
import email
from email.header import decode_header
import base64
from bs4 import BeautifulSoup
import re

def read(sender_of_interest=None):
    # Очищаем файл перед записью
    with open('filtered_emails.txt', 'r+') as f:
        f.truncate(0) 

    # Подключаемся к серверу
    imap = imaplib.IMAP4_SSL("imap.gmail.com", 993)
    imap.login('remboplas@gmail.com', 'trok ekct masw xlwz')
    imap.select('INBOX')

    # Поиск всех писем
    if sender_of_interest:
        status, response = imap.uid('search', None, 'ALL', 'FROM {0}'.format(sender_of_interest))
    else:
        status, response = imap.uid('search', None, 'ALL')

    if status == 'OK':
        unread_msg_nums = response[0].split()
        print(f"Found {len(unread_msg_nums)} emails")  # Отладочная информация
    else:
        unread_msg_nums = []
        print("No emails found")

    # Инвертируем список ID, чтобы получить последние письма первыми
    unread_msg_nums = unread_msg_nums[::-1]

    data_list = []
    count = 0  # Счётчик писем

    for e_id in unread_msg_nums:
        if count >= 10:  # Ограничение на 10 последних писем
            break

        e_id = e_id.decode('utf-8')
        _, response = imap.uid('fetch', e_id, '(RFC822)')

        # Используем message_from_bytes для правильного парсинга писем
        raw_email = response[0][1]
        email_message = email.message_from_bytes(raw_email)

        # Проверка, multipart или нет, чтобы обработать текстовые и HTML части
        if email_message.is_multipart():
            for part in email_message.walk():
                content_type = part.get_content_type()
                if content_type == 'text/plain':  # Извлекаем только текстовую часть
                    body = part.get_payload(decode=True).decode('utf-8', errors='ignore')
                    data_list.append(body)
                    print(f"Added text/plain part from email {e_id}")  # Отладочная информация
                    count += 1
                    break  # Если нашли нужную часть, выходим
        else:
            # Если не multipart, просто получаем тело письма
            body = email_message.get_payload(decode=True).decode('utf-8', errors='ignore')
            data_list.append(body)
            print(f"Added non-multipart email {e_id}")  # Отладочная информация
            count += 1

    # Запись в файл
    with open('filtered_emails.txt', 'w', encoding='utf-8') as f:
        for elem in data_list:
            # Удаляем блоки стилей <style>...</style>
            elem = re.sub(r"<style.*?>.*?</style>", "", elem, flags=re.S)
            elem = re.sub(r"&nbsp;", "", elem, flags=re.S)
            # Удаляем все HTML-теги
            elem = re.sub(r"<[^>]+>", "", elem, flags=re.S)
            
            # Разбиваем на строки и записываем строки, если они не пустые
            for line in elem.splitlines():
                if line.strip():  # Проверяем, не пустая ли строка
                    f.write(line + '\n')  # Записываем в файл
            
            # Добавляем разделитель для каждого письма
            f.write('\n++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++\n')

    # Завершаем работу с сервером
    imap.logout()

# Вызов функции
read()


Found 1497 emails
Added text/plain part from email 4783
Added text/plain part from email 4782
Added text/plain part from email 4781
Added text/plain part from email 4780
Added text/plain part from email 4779
Added text/plain part from email 4778
Added text/plain part from email 4777
Added text/plain part from email 4776
Added text/plain part from email 4775
Added non-multipart email 4774
